# NLP 191 - Assignment 1
### Dolav Nitay - 203598990

In [2]:
from nltk.tokenize import TreebankWordTokenizer

In [25]:
def isNumber(input: str) -> bool:
    try:
        int(input)
        return True
    except ValueError:
        return False

In [27]:
s = "Give me 500$"
tokens = TreebankWordTokenizer().tokenize(s)
for ind, token in enumerate(tokens):
    if isNumber(token):
        tokens[ind] = 'N'
print(tokens)

['Give', 'me', 'N', '$']
